# Red Neuronal estilo StatQuest (scikit-learn)

**Objetivo:** Reproducir el ejemplo clásico de StatQuest: una red neuronal pequeñita que aprende una *curva* (la "green squiggle") para mapear **dosis → efectividad**.

**Arquitectura:**
- Entrada: 1 característica (dosis ∈ [0, 1])
- Capa oculta: 2 neuronas (activación ReLU)
- Salida: 1 valor (efectividad)

Usaremos `MLPRegressor` de `scikit-learn` y visualizaremos la curva aprendida.

## 1) Importaciones
Solo necesitamos NumPy, Matplotlib (para graficar) y `MLPRegressor` de scikit-learn.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPRegressor

np.set_printoptions(precision=4, suppress=True)

## 2) Conjunto de datos de juguete
Codificamos la idea simple: **dosis baja → 0**, **dosis media → 1**, **dosis alta → 0**.

In [ ]:
# X: dosis (entradas); y: efectividad (etiquetas/objetivo)
X = np.array([[0.0], [0.5], [1.0]])
y = np.array([0.0, 1.0, 0.0])
X, y

## 3) Definir la red neuronal
- **`hidden_layer_sizes=(2,)`** → una capa oculta con 2 neuronas.
- **`activation='relu'`** → línea "doblada" muy usada en la práctica.
- **`solver='lbfgs'`** → robusto para conjuntos de datos muy pequeños.
- **`max_iter=10000`** → suficientes iteraciones para asegurar convergencia.

In [ ]:
model = MLPRegressor(hidden_layer_sizes=(2,),
                     activation='relu',
                     solver='lbfgs',
                     max_iter=10000,
                     random_state=42)
model

## 4) Entrenar el modelo
Backpropagation corre "por debajo" para ajustar **pesos** y **sesgos** minimizando el error.

In [ ]:
model.fit(X, y)
print("Pérdida tras converger:", getattr(model, 'loss_', None))

## 5) Predicción en una malla densa para ver la *curva*
Evaluamos de 0 a 1 y graficamos el mapeo aprendido. **Nota:** Usamos Matplotlib, un solo gráfico y sin fijar colores explícitos.

In [ ]:
X_test = np.linspace(0, 1, 201).reshape(-1, 1)
y_pred = model.predict(X_test)

plt.figure(figsize=(8,5))
plt.scatter(X, y, label='Datos reales')
plt.plot(X_test, y_pred, linewidth=2, label='Red neuronal (curva)')
plt.title("Ajuste de una 'curva' con Red Neuronal (estilo StatQuest)")
plt.xlabel("Dosis (0–1)")
plt.ylabel("Efectividad (0–1)")
plt.legend()
plt.grid(True)
plt.show()

## 6) Mirada interna: pesos y sesgos aprendidos
Análogo a pendiente e intercepto en regresión lineal, pero para cada conexión/nodo.

In [ ]:
print("Pesos: entrada → oculta (shape {}):\n".format(model.coefs_[0].shape), model.coefs_[0])
print("\nSesgos: capa oculta (len {}):\n".format(len(model.intercepts_[0])), model.intercepts_[0])
print("\nPesos: oculta → salida (shape {}):\n".format(model.coefs_[1].shape), model.coefs_[1])
print("\nSesgo: salida (len {}):\n".format(len(model.intercepts_[1])), model.intercepts_[1])

## 7) Predicción puntual y regla de decisión
Como en el video, probamos con **dosis = 0.5**. Si el valor está más cerca de 1 que de 0, lo interpretamos como **“efectivo”**.

In [ ]:
dose = np.array([[0.5]])
pred = model.predict(dose)[0]
print(f"Predicción para dosis=0.5: {pred:.4f}")
print("Interpretación:", "Efectivo" if abs(pred-1.0) < abs(pred-0.0) else "No efectivo")

## 8) (Opcional) Explora otras activaciones y anchos
Prueba cambiar `activation` (`'tanh'`, `'relu'`, `'logistic'`) y `hidden_layer_sizes` (por ejemplo, `(5,)`, `(10,)`), y vuelve a entrenar y graficar para ver cómo cambia la curva. Redes más profundas y anchas representan formas más complejas, pero ¡cuidado con **overfitting** en un dataset tan pequeño!